# Logistic and SVM Model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
import xgboost as xgb
import sklearn.datasets
import gc
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_tiny_X = pd.read_csv('music_data/train_tiny_X.csv')
train_tiny_y = pd.read_csv('music_data/train_tiny_Y.csv')

In [4]:
display(train_tiny_y)

,Unnamed: 0,target
0,2942719,0.0
1,4875524,0.0
2,6589819,0.0
3,1172060,1.0
4,2069395,0.0
5,4966246,0.0
6,69897,1.0
7,933241,1.0
8,1122481,1.0
9,2680592,0.0


In [5]:
train_X = pd.read_csv('music_data/train_X.csv')
val_X = pd.read_csv('music_data/valid_X.csv')
train_y = pd.read_csv('music_data/train_Y.csv')
val_y = pd.read_csv('music_data/valid_Y.csv')

In [6]:
val_tiny_X = val_X[:1000]
val_tiny_y = val_y[:1000]

In [7]:
catCols = ['msno', 'song_id', 'source_screen_name', 'source_system_tab', 'source_type', 'genre_ids', 'artist_name', 
           'composer', 'lyricist', 'language', 'city', 'gender', 'registered_via']
numCols = ['bd', 'song_length']

In [8]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4426382 entries, 0 to 4426381
Data columns (total 19 columns):
Unnamed: 0                int64
msno                      int64
song_id                   int64
source_screen_name        int64
source_system_tab         int64
source_type               int64
song_length               float64
genre_ids                 int64
artist_name               int64
composer                  int64
lyricist                  int64
language                  int64
city                      int64
bd                        int64
gender                    int64
registered_via            int64
registration_init_time    object
expiration_date           object
time                      float64
dtypes: float64(2), int64(15), object(2)
memory usage: 641.6+ MB


In [26]:
print(np.sum(train_X.isnull()))
print(train_X.shape)

Unnamed: 0                0
msno                      0
song_id                   0
source_screen_name        0
source_system_tab         0
source_type               0
song_length               0
genre_ids                 0
artist_name               0
composer                  0
lyricist                  0
language                  0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
expiration_date           0
time                      0
dtype: int64
(4426382, 19)


In [11]:
all_train = pd.concat([train_X, train_y])
all_train.shape

/home/cs229/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(8852764, 20)

In [12]:
np.sum(train_y.isnull())

Unnamed: 0    0
target        0
dtype: int64

In [13]:
train_y.head(100)

,Unnamed: 0,target
0,2942719,0.0
1,4875524,0.0
2,6589819,0.0
3,1172060,1.0
4,2069395,0.0
5,4966246,0.0
6,69897,1.0
7,933241,1.0
8,1122481,1.0
9,2680592,0.0


In [14]:
all_train.dropna(axis=0, how='any').shape

(0, 20)

## Logistic Regression

In [15]:
# numRow = len(all_data)
# def checkCol(df, colName, numRow, isCata = True):
#     print(colName, ':')
#     print('Any NA?: ', df[colName].isnull().values.any())
#     if df[colName].isnull().values.any():
#         numNA = sum(df[colName].isnull().values)
#         print ('    # of NA: ', numNA)
#         print ('    NA%:     ', numNA/numRow)
#     if isCata:
#         levelList = df[colName].unique()
#         print('Different levels: ', len(levelList), levelList)
#         fillna(df, colName, isCata = True)
        
#     else:
#         print('range: ', min(df[colName].astype(float).dropna()), max(df[colName].astype(float).dropna()))
#         fillna(df, colName, isCata = False)
        
# def fillna(df, colName, isCata = True):
#     if isCata:
#         df[colName] = df[colName].fillna('0')
#     else:
#         if df[colName].isnull().values.any():
#             df[colName+'_dm'] = df[colName].isnull().astype(int)
#         df[colName] = df[colName].fillna(0)
#     return df

In [17]:
#debugging with tiny data set
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

clf.fit(train_tiny_X, train_tiny_y['target'])
print("Model score on training set:")
print("model score: %.3f" % clf.score(train_tiny_X, train_tiny_y['target']))
print(classification_report(train_tiny_y['target'], clf.predict(train_tiny_X)))
print(roc_auc_score(train_tiny_y['target'], clf.predict_proba(train_tiny_X)[:,1]))

/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.633
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      4985
         1.0       0.93      0.94      0.93      5015

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

0.9814643531791786


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [18]:
print("Model score on validation set:")
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf.predict(val_tiny_X)))
print(roc_auc_score(val_tiny_y['target'], clf.predict_proba(val_tiny_X)[:,1]))

model score: 0.633
              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62       477
         1.0       0.65      0.65      0.65       523

   micro avg       0.63      0.63      0.63      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.63      0.63      0.63      1000

0.6737737051601188


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [46]:
# DON'T RUN IT YET!!!!!! HUGE DATASET!!!

# numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
# categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
# preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
#                                                ('cat', categorical_transformer, catCols)])
# clf = Pipeline(steps=[('preprocessor', preprocessor),
#                       ('classifier', LogisticRegression())])

# clf.fit(train_X, train_y['target'])
# print("model score: %.3f" % clf.score(val_X, val_y['target']))
# print(classification_report(val_y['target'], clf.predict(val_X)))
# print(roc_auc_score(val_y, clf.predict_proba(val_X)['target']))

Default: Logistic Regression with No Penalty, learning rate = 'optimal'  
val acc = 0.66

In [62]:
#mini batch training with full data and early stopping
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf_log = Pipeline(steps=[('preprocessor', preprocessor),
                ('classifier', SGDClassifier(loss='log', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                max_iter=1000, tol=0.001, shuffle=True, verbose=1, epsilon=0.1, n_jobs=None, 
                                random_state=None, learning_rate='adaptive', eta0=0.0001, 
                                power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                class_weight=None, warm_start=True, average=False))])
#default, no penalty
clf_log.fit(train_X, train_y['target'])
print("model score: %.3f" % clf_log.score(val_X, val_y['target']))
print(classification_report(val_y['target'], clf_log.predict(val_X)))
print(roc_auc_score(val_y['target'], clf_log.predict_proba(val_X)[:,1]))
#val acc 0.660


-- Epoch 1
Norm: 249.28, NNZs: 422416, Bias: -0.009009, T: 3983743, Avg. loss: 0.753278
Total training time: 3.08 seconds.
-- Epoch 2
Norm: 243.67, NNZs: 422416, Bias: -0.008884, T: 7967486, Avg. loss: 0.660473
Total training time: 7.29 seconds.
-- Epoch 3
Norm: 240.69, NNZs: 422416, Bias: -0.009143, T: 11951229, Avg. loss: 0.646121
Total training time: 11.93 seconds.
-- Epoch 4
Norm: 238.69, NNZs: 422416, Bias: -0.009061, T: 15934972, Avg. loss: 0.638001
Total training time: 16.43 seconds.
-- Epoch 5
Norm: 237.21, NNZs: 422416, Bias: -0.009070, T: 19918715, Avg. loss: 0.632496
Total training time: 20.93 seconds.
-- Epoch 6
Norm: 236.03, NNZs: 422416, Bias: -0.008970, T: 23902458, Avg. loss: 0.628386
Total training time: 25.75 seconds.
Convergence after 6 epochs took 26.81 seconds
model score: 0.660
              precision    recall  f1-score   support

         0.0       0.66      0.64      0.65    732500
         1.0       0.66      0.68      0.67    742961

   micro avg       0.66  

In [13]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf_svm = Pipeline(steps=[('preprocessor', preprocessor),
                ('classifier', SGDClassifier(loss='hinge', penalty=None, early_stopping=True, 
                                             validation_fraction=0.1, n_iter_no_change=5))])
#default, no penalty
clf_svm.fit(train_X, train_y['target'])
print("model score: %.3f" % clf_svm.score(val_X, val_y['target']))
print(classification_report(val_y['target'], clf_svm.predict(val_X)))
print(roc_auc_score(val_y['target'], clf_svm.predict(val_X)>0.5))

model score: 0.668
              precision    recall  f1-score   support

         0.0       0.67      0.66      0.66    732500
         1.0       0.67      0.68      0.67    742961

   micro avg       0.67      0.67      0.67   1475461
   macro avg       0.67      0.67      0.67   1475461
weighted avg       0.67      0.67      0.67   1475461

0.6676017828385639


best SVM model(with no penalty):  
learning rate = 'adaptive', eta = 0.01  
best val acc = 0.697

In [10]:
etas = [0.01]
best_model_svm = None
best_model_score_svm = 0.0
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
for eta in etas:
    clf_svm = Pipeline(steps=[('preprocessor', preprocessor),
                    ('classifier', SGDClassifier(loss='hinge', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                    max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                                    random_state=None, learning_rate='adaptive', eta0=eta, 
                                    power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                    class_weight=None, warm_start=True, average=False))])
    clf_svm.fit(train_X, train_y['target'])
    model_score = clf_svm.score(val_X, val_y['target'])
    if model_score > best_model_score_svm:
        best_model_score_svm = model_score
        best_model_svm = clf_svm
        print('eta0:',eta)
        print('best val acc:',best_model_score_svm)
                                        
# print("best model score:" ,best_model_score_svm)
# print(classification_report(val_y['target'], best_model_svm.predict(val_X)))
# print(roc_auc_score(val_y['target'], best_model_svm.predict(val_X)>0.5))
#print(roc_auc_score(val_y['target'], clf_svm.predict_proba(val_X)[:,1]))

eta0: 0.01
best val acc: 0.696620920512301


In [11]:
print("best model score:" ,best_model_score_svm)
print(classification_report(val_y['target'], best_model_svm.predict(val_X)))
print(roc_auc_score(val_y['target'], best_model_svm.predict(val_X)>0.5))

best model score: 0.696620920512301
              precision    recall  f1-score   support

         0.0       0.69      0.70      0.70    732500
         1.0       0.70      0.70      0.70    742961

   micro avg       0.70      0.70      0.70   1475461
   macro avg       0.70      0.70      0.70   1475461
weighted avg       0.70      0.70      0.70   1475461

0.696625487436529


best logistic regression model(with no penalty):  
learning rate = 'adaptive', eta = 0.01  
best val acc = 0.694

In [69]:
#fine tune learning rate on logistic regression
best_model_df = None
best_model_score_df = 0.0
learning_rate = ['optimal','invscaling','adaptive']
etas = [0.01,0.001,0.0001]
for lr in learning_rate:
    for eta in etas:
        clf_log_df = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', SGDClassifier(loss='log', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                        max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                                        random_state=None, learning_rate=lr, eta0=eta, 
                                        power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                        class_weight=None, warm_start=True, average=False))])
        clf_log_df.fit(train_X, train_y['target'])
        model_score = clf_log_df.score(val_X, val_y['target'])
        if model_score > best_model_score_df:
            best_model_score_df = model_score
            best_model_df = clf_log_df
            print('learning_rate:',lr,'eta0:',eta)
            print('best val acc:',best_model_score_df)
                                        
print("best model score:" ,best_model_score_df)
print(classification_report(val_y['target'], best_model_df.predict(val_X)))
print(roc_auc_score(val_y['target'], best_model_df.predict_proba(val_X)[:,1]))

learning_rate: optimal eta0: 0.01
best val acc: 0.6600669214570903
learning_rate: optimal eta0: 0.0001
best val acc: 0.6603908880004283
learning_rate: adaptive eta0: 0.01
best val acc: 0.694752351976772
best model score: 0.694752351976772
              precision    recall  f1-score   support

         0.0       0.69      0.69      0.69    732500
         1.0       0.70      0.70      0.70    742961

   micro avg       0.69      0.69      0.69   1475461
   macro avg       0.69      0.69      0.69   1475461
weighted avg       0.69      0.69      0.69   1475461

0.763170012584044


In [72]:
#fine tune SVM and Logistic regression with penalty
best_model = None
best_model_score = 0.0
loss = ['log', 'hinge']
penalty = ['l1','l2','elasticnet']

for ls in loss:
    for plty in penalty: 
        clf_log_tune = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', SGDClassifier(loss=ls, penalty=plty, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                        max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, 
                                        random_state=None, learning_rate='adaptive', eta0=0.01, 
                                        power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, 
                                        class_weight=None, warm_start=True, average=False))])
        clf_log_tune.fit(train_X, train_y['target'])
        model_score = clf_log_tune.score(val_X, val_y['target'])
        print( 'loss:', ls, 'penalty:', plty,'val acc:', model_score)
        if model_score > best_model_score:
            best_model_score = model_score
            best_model = clf_log_tune
            print( 'best loss:', ls, 'best penalty:', plty,'best val acc:', model_score)

print("best model score:" ,best_model_score)
print(classification_report(val_y['target'], best_model.predict(val_X)))
print(roc_auc_score(val_y['target'], best_model.predict_proba(val_X)[:,1]))

loss: log penalty: l1 val acc: 0.6377667725544762
best loss: log best penalty: l1 best val acc: 0.6377667725544762
loss: log penalty: l2 val acc: 0.6505539624564797
best loss: log best penalty: l2 best val acc: 0.6505539624564797
loss: log penalty: elasticnet val acc: 0.644621579289456
loss: hinge penalty: l1 val acc: 0.6365359707915018
loss: hinge penalty: l2 val acc: 0.6395126675662725
loss: hinge penalty: elasticnet val acc: 0.6382168013929206
best model score: 0.6505539624564797
              precision    recall  f1-score   support

         0.0       0.66      0.61      0.63    732500
         1.0       0.64      0.69      0.66    742961

   micro avg       0.65      0.65      0.65   1475461
   macro avg       0.65      0.65      0.65   1475461
weighted avg       0.65      0.65      0.65   1475461

0.7114225593584986


# SVM with kernel

In [1]:
#rbf
from sklearn.kernel_approximation import RBFSampler
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
rbf_feature = RBFSampler(gamma=0.2, random_state=1)
clf_kernel = Pipeline(steps=[('preprocessor', preprocessor),
                ('kernel',rbf_feature),
                ('classifier', SGDClassifier(loss='hinge', penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                                max_iter=1000, tol=0.001, shuffle=True, verbose=1, epsilon=0.1, n_jobs=None, 
                                random_state=None, learning_rate='adaptive', eta0=0.01, 
                                power_t=0.5, early_stopping=True, validation_fraction=0.1, n_iter_no_change=5))])
#default, no penalty
clf_kernel.fit(train_tiny_X, train_tiny_y['target'])
print("model score: %.3f" % clf_kernel.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf_kernel.predict(val_tiny_X)))
#print(roc_auc_score(val_tiny_y['target'], clf_kernel.predict_proba(val_tiny_X)[:,1]))



NameError: name 'Pipeline' is not defined

## Logistic Regression with Regularization

### L1

In [19]:
from sklearn.feature_selection import SelectKBest
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(penalty='l1'))])

clf.fit(train_tiny_X, train_tiny_y['target'])
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(train_tiny_y['target'], clf.predict(train_tiny_X)))
print(roc_auc_score(train_tiny_y['target'], clf.predict_proba(train_tiny_X)[:,1]))

/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.660
              precision    recall  f1-score   support

         0.0       0.72      0.69      0.71      4985
         1.0       0.71      0.73      0.72      5015

   micro avg       0.71      0.71      0.71     10000
   macro avg       0.71      0.71      0.71     10000
weighted avg       0.71      0.71      0.71     10000

0.7882900546104915


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [20]:
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf.predict(val_tiny_X)))
print(roc_auc_score(val_tiny_y['target'], clf.predict_proba(val_tiny_X)[:,1]))

model score: 0.660
              precision    recall  f1-score   support

         0.0       0.64      0.65      0.65       477
         1.0       0.68      0.67      0.67       523

   micro avg       0.66      0.66      0.66      1000
   macro avg       0.66      0.66      0.66      1000
weighted avg       0.66      0.66      0.66      1000

0.6840073595728562


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


## L2

In [21]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(penalty='l2'))])

clf.fit(train_tiny_X, train_tiny_y['target'])
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(train_tiny_y['target'], clf.predict(train_tiny_X)))
print(roc_auc_score(train_tiny_y['target'], clf.predict_proba(train_tiny_X)[:,1]))

/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


model score: 0.633
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      4985
         1.0       0.93      0.94      0.93      5015

   micro avg       0.93      0.93      0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

0.9814643531791786


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [22]:
print("model score: %.3f" % clf.score(val_tiny_X, val_tiny_y['target']))
print(classification_report(val_tiny_y['target'], clf.predict(val_tiny_X)))
print(roc_auc_score(val_tiny_y['target'], clf.predict_proba(val_tiny_X)[:,1]))

model score: 0.633
              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62       477
         1.0       0.65      0.65      0.65       523

   micro avg       0.63      0.63      0.63      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.63      0.63      0.63      1000

0.6737737051601188


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


## Gradient Boosting & Decision Tree
Moved to "Tree & Boosting" Notebook

## Lasso

In [26]:
from sklearn.linear_model import Lasso
alphas = [0.01,0.05,0.1,0.2]
for a in alphas:
    print('alpha: {}'.format(a))
    lasso = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor',  Lasso(alpha = a))])
    lasso.fit(train_tiny_X, train_tiny_y['target'])
    print(np.sqrt(metrics.mean_squared_error(val_tiny_y['target'], lasso.predict(val_tiny_X))))

alpha: 0.01


/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype i

0.48453905786700674
alpha: 0.05


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


0.49497671219414635
alpha: 0.1


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/cs229/.local/lib/python3.5/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


0.49993324554384255
alpha: 0.2
0.49993324554384255


/home/cs229/.local/lib/python3.5/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [ ]:
gc.collect()